In [9]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression

In [7]:
des_concat_train = np.load('./features/descriptions_train_concat_embed_vectors.npy')
des_concat_test = np.load('./features/descriptions_test_concat_embed_vectors.npy')
des_long_train = np.load('./features/descriptions_train_long_embed_vectors.npy')
des_long_test = np.load('./features/descriptions_test_long_embed_vectors.npy')
image_1000_train = np.load('./features/image_features_1000_train.npy')
image_1000_test = np.load('./features/image_features_1000_test.npy')
image_2048_train = np.load('./features/image_features_2048_train.npy')
image_2048_test = np.load('./features/image_features_2048_test.npy')

IOError: Failed to interpret file './features/descriptions_train_concat_embed_vectors.npy' as a pickle

In [8]:
def get_nearest_20(vec, vec_set):
    dis = [np.linalg.norm(np.array(vec) - np.array(x)) for x in vec_set]
    return np.argsort(dis)[:20]

def map20score(y, pred):
    if y in pred:
        return (20 - pred.tolist().index(y)) / 20.0
    else:
        return 0

def map20eval(pls, des_vectors, image_vectors):
    image_pred = pls.predict(des_vectors)
    top_20 = [get_nearest_20(vec, image_vectors) for vec in image_pred]
    print(len(top_20))
    scores = [map20score(i, top_20[i]) for i in range(len(top_20))]
    print "score: %f" % np.mean(scores)
    return scores

In [4]:
pls_concat_1000 = PLSRegression(n_components=256)
pls_concat_1000.fit(des_concat_train, image_1000_train)

/anaconda2/lib/python2.7/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [6]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]

In [12]:
def output_submission(top_20, output_path):
    top_20_image_IDs = map(lambda x: ' '.join(map(lambda xx: str(xx) + '.jpg', x)), top_20)
    description_ID = map(lambda x: str(x) + '.txt', range(len(top_20)))
    submission_df = pd.DataFrame({'Descritpion_ID': description_ID, 'Top_20_Image_IDs': top_20_image_IDs})
    submission_df.to_csv(output_path, index=False)


In [13]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000.csv')

In [14]:
pls_concat_1000 = PLSRegression(n_components=256, max_iter=1000)
pls_concat_1000.fit(des_concat_train, image_1000_train)

PLSRegression(copy=True, max_iter=1000, n_components=256, scale=True,
       tol=1e-06)

In [15]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]


In [16]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000_1000iter.csv')

In [40]:
des_BOW_noun_train = np.load('features/descriptions_train_BOW_noun.npy')
des_BOW_noun_test = np.load('features/descriptions_test_BOW_noun.npy')

In [56]:
pls_BOW_noun_1000 = PLSRegression(n_components=10)
pls_BOW_noun_1000.fit(des_BOW_noun_train[:8000], image_1000_train[:8000])
print pls_BOW_noun_1000.score(des_BOW_noun_train[8000:], image_1000_train[8000:])

0.3791097044896606


In [49]:
scores = map20eval(pls_BOW_noun_1000, des_BOW_noun_train[8000:], image_1000_train[8000:])

2000
score: 0.290600


In [51]:
pls_BOW_noun_1000 = PLSRegression(n_components=256)
pls_BOW_noun_1000.fit(des_BOW_noun_train, image_1000_train)

PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [52]:
pred_BOW_noun_1000 = pls_BOW_noun_1000.predict(des_BOW_noun_test)
top_20_BOW_noun_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_BOW_noun_1000]

In [53]:
output_submission(top_20_BOW_noun_1000, 'submissions/PLSR_256_BOW_noun_1000.csv')

In [59]:
des_BOW_noun_train = np.load('features/descriptions_train_BOW_noun_5452.npy')
des_BOW_noun_test = np.load('features/descriptions_test_BOW_noun_5452.npy')

In [60]:
from sklearn.decomposition import PCA

In [89]:
pca = PCA(n_components=512)

In [90]:
pca.fit(des_BOW_noun_train)

PCA(copy=True, iterated_power='auto', n_components=512, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [91]:
des_BOW_noun_train_pca = pca.transform(des_BOW_noun_train)
des_BOW_noun_test_pca = pca.transform(des_BOW_noun_test)

In [92]:
print des_BOW_noun_train_pca.shape

(10000, 512)


In [93]:
pls_BOW_noun_1000 = PLSRegression(n_components=20)
pls_BOW_noun_1000.fit(des_BOW_noun_train_pca[:8000], image_1000_train[:8000])
print pls_BOW_noun_1000.score(des_BOW_noun_train_pca[8000:], image_1000_train[8000:])

0.43198989225672163


In [96]:
pls_BOW_noun_1000 = PLSRegression(n_components=256)
pls_BOW_noun_1000.fit(des_BOW_noun_train_pca, image_1000_train)

PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [97]:
pred_BOW_noun_1000_5452 = pls_BOW_noun_1000.predict(des_BOW_noun_test_pca)
top_20_BOW_noun_1000_5452 = [get_nearest_20(vec, image_1000_test) for vec in pred_BOW_noun_1000_5452]

In [99]:
output_submission(top_20_BOW_noun_1000_5452452, 'submissions/PLSR_256_BOW_noun_1000_5452_pca_512.csv')

In [22]:
"""
Luffy part.
"""

import pickle
import os.path
from sklearn.decomposition import PCA

train_description_feature_map = pickle.load(open('./features/11_30_[train]_description_normalized_only_>1_vector_py27.pkl', 'rb'))
test_derscription_feature_map = pickle.load(open('./features/11_30_[test]_description_normalized_only_>1_vector_py27.pkl', 'rb'))

train_bow_lst = list(map(lambda x: x["BOW_all_normalized_vector"], train_description_feature_map))
test_bow_lst = list(map(lambda x: x["BOW_all_normalized_vector"], test_derscription_feature_map))

pca_model_name = "./models/pca_bow_all_to_1024.pkl"
if os.path.exists(pca_model_name):
    pca = pickle.load(open(pca_model_name, 'rb'))
else:
    pca = PCA(n_components=3500)
    pca.fit(train_bow_lst)
    pickle.dump(pca, open(pca_model_name, 'wb'), protocol=2)

des_BOW_all_train_pca = pca.transform(train_bow_lst)
des_BOW_all_test_pca = pca.transform(test_bow_lst)

print des_BOW_all_test_pca.shape

/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator PCA from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(2000, 1024)


In [ ]:
train_img_map = pickle.load(open('./features/12_1_train_img_feature_map_py27.pkl', 'rb'))
test_img_map = pickle.load(open('./features/12_1_test_img_feature_map_py27.pkl', 'rb'))

In [19]:
train_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], train_img_map)))
test_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], test_img_map)))

(10000, 2048)


In [24]:
model_name = "./models/pls_600_pool5_to_BOW_all_pca_1024.pkl"
if os.path.exists(model_name):
    model = pickle.load(open(model_name, 'rb'))
else:
    model = PLSRegression(n_components=800)
    model.fit(train_pool5_img[:8000], des_BOW_all_train_pca[:8000])
    pickle.dump(model, open(model_name, 'wb'), protocol=2)
    print model.score(train_pool5_img[8000:], des_BOW_all_train_pca[8000:])
print "model loaded..."

model loaded...


/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator PLSRegression from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [30]:
from scipy.spatial.distance import cdist
import csv

pred_test_pool5 = model.predict(test_pool5_img)

dist = cdist(des_BOW_all_test_pca, pred_test_pool5, 'cosine')
print("description * images dist matrix, shape:", dist.shape)
sorted_id = np.argsort(dist) # dist: N_description * N_images dist matrix.

with open('./submission/submission_plsr_600_pool5_to_bow_all_pca_1024.csv', 'w') as csvfile:
        # write csv header
        fieldnames = ['Descritpion_ID', "Top_20_Image_IDs"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for i, row in enumerate(sorted_id):
            top_choices =  list(map(lambda x: str(x) + ".jpg", row[:20]))
            res = {}
            res['Descritpion_ID'] = str(i) + ".txt" # file name
            res['Top_20_Image_IDs'] = " ".join(top_choices)
            writer.writerow(res)

print("Writing Complete.")

('description * images dist matrix, shape:', (2000, 2000))
Writing Complete.
